## Ask the Prince
---

This notebook will allow us to ask questions to LLM that has The Prince by Machiavelli as contextt

### Importing Libaries

In [1]:
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [3]:
import dotenv
import os

# Load variables from the .env file
dotenv.load_dotenv(dotenv.find_dotenv("env"))
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

Setting API KEY

### Loading the book

In [4]:
the_prince_path = "../../data/Machiavelli/the_prince.txt"

In [5]:
with open(the_prince_path, "r") as file:
    the_prince = file.read()

print(the_prince[:300])

The Project Gutenberg eBook of The Prince
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License includ


### Text Splitter
---

The whole text is too long to embed at the same time, so we will split it in chunks

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Text splitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 500,
    chunk_overlap = 20
)

texts = text_splitter.create_documents([the_prince])
print(len(texts))

793


In [ ]:
texts[:200] # Looking at small portion of the book

### Embeeding the book
---

Using OpenAI Embeddings

In [8]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [9]:
embeddings_list = embeddings.embed_documents([text.page_content for text in texts])

In [10]:
print(f"You have {len(embeddings_list)} embeddings")
print(f" Here's a sample of one: {embeddings_list[0][:3]}...")

You have 793 embeddings
 Here's a sample of one: [-0.002616283325341809, -0.020187786422823248, -0.012890996777875878]...


In [11]:
print(len(embeddings_list[0]))

1536


Saving the embeddings

In [12]:
import pickle

# Assuming you have already generated the embeddings_list
# ...

# Specify the file path where you want to save the embeddings_list
file_path = "../../data/Machiavelli/the_prince_embedded.pkl"

# Open the file in binary write mode
with open(file_path, "wb") as file:
    # Serialize and save the embeddings_list to the file
    pickle.dump(embeddings_list, file)

print(f"Embeddings list saved to {file_path}")


Embeddings list saved to ../../data/Machiavelli/the_prince_embedded.pkl


In [13]:
# Loading the embeddings
# To load the saved embeddings_list from the file
with open(file_path, "rb") as file:
    loaded_embeddings_list = pickle.load(file)

# Now, loaded_embeddings_list contains the data from the saved file
print(loaded_embeddings_list[0][:6])

[-0.002616283325341809, -0.020187786422823248, -0.012890996777875878, -0.04559853802597419, 0.006077458323439668, 0.03745685624260947]


### Creating Vectorstore

In [14]:
from langchain.vectorstores import FAISS
the_prince_vectorstore = FAISS.from_documents(texts, embedding=embeddings)

In [15]:
# Saving it
the_prince_vectorstore.save_local(folder_path="../../data/Machiavelli/", index_name="the_prince_vectorstore")

#### Questions
---

To assess how well a language model (LLM) understands the context of Machiavelli's book "The Prince" and can answer questions as if it were "The Prince," you can ask a range of questions that cover various aspects of the book's themes, philosophy, and content. Here are some example questions you can ask, along with suggestions on how to score the responses:

1. **Historical Context**:
   - Question: "Can you provide some historical background on the political situation during the time when 'The Prince' was written?"
   - Scoring: A good response should mention the time period (Renaissance Italy), the political turmoil, and perhaps specific historical events. A comprehensive answer would receive a higher score.

2. **Central Themes**:
   - Question: "What are the central themes and ideas presented in 'The Prince'?"
   - Scoring: A good response should mention key themes like power, leadership, morality, and realpolitik. A response that elaborates on these themes and their implications would score higher.

3. **Machiavellian Principles**:
   - Question: "What are some of the principles advocated by Machiavelli in 'The Prince'?"
   - Scoring: A comprehensive response should include concepts like virtù, fortuna, the ends justifying the means, and the importance of effective statecraft. A response that provides examples from the book would score higher.

4. **Advice to Rulers**:
   - Question: "What advice does Machiavelli give to rulers in terms of maintaining power and stability?"
   - Scoring: A good response should mention strategies such as being feared vs. loved, maintaining a strong army, and adapting to changing circumstances. Specific advice from the book would earn a higher score.

5. **Morality vs. Pragmatism**:
   - Question: "How does 'The Prince' address the conflict between moral principles and political pragmatism?"
   - Scoring: A thoughtful response should discuss the tension between moral values and political necessity, perhaps citing examples from the book. An in-depth analysis would score higher.

6. **Influence on Political Thought**:
   - Question: "What impact has 'The Prince' had on political thought and practice throughout history?"
   - Scoring: A good response should mention the book's influence on realpolitik, political realism, and its controversial reputation. Providing specific historical examples would enhance the score.

7. **Critiques and Controversies**:
   - Question: "What are some common criticisms and controversies surrounding 'The Prince'?"
   - Scoring: A comprehensive response should mention criticisms related to Machiavellian ethics and the book's perceived endorsement of ruthless behavior. Discussing differing interpretations would score higher.

8. **Applicability Today**:
   - Question: "In what ways do you think the principles discussed in 'The Prince' are still relevant in modern politics and leadership?"
   - Scoring: A good response should identify contemporary examples or applications of Machiavellian principles in politics, business, or leadership. The ability to draw parallels would score higher.

Scoring should be based on the depth, accuracy, and relevance of the responses. You can assign scores on a scale (e.g., 1 to 5) and evaluate each response accordingly. A well-structured and insightful answer that demonstrates a deep understanding of "The Prince" and its context should receive a higher score, while less comprehensive or inaccurate responses should receive lower scores.


In [16]:
questions = [
    "Can you provide some historical background on the political situation during the time when 'The Prince' was written?",
    "What are the central themes and ideas presented in 'The Prince'?",
    "What are some of the principles advocated by Machiavelli in 'The Prince'?",
    "What advice does Machiavelli give to rulers in terms of maintaining power and stability?",
    "How does 'The Prince' address the conflict between moral principles and political pragmatism?",
    "What impact has 'The Prince' had on political thought and practice throughout history?",
    "What are some common criticisms and controversies surrounding 'The Prince'?",
    "In what ways do you think the principles discussed in 'The Prince' are still relevant in modern politics and leadership?"
]

### Chain

In [17]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [18]:
chain = load_qa_chain(
    llm = OpenAI(
        model_name="text-davinci-003",
        temperature=0
        ),
    chain_type="stuff",
    verbose=False
)

In [19]:
import time

for question in questions:
    print("------"*20)
    print("Question:", question)
    docs = the_prince_vectorstore.similarity_search(question)
    output = chain.run(input_documents = docs, question=question)
    print("LMM Output:")
    print(str.strip(output))
    time.sleep(30)



------------------------------------------------------------------------------------------------------------------------
Question: Can you provide some historical background on the political situation during the time when 'The Prince' was written?
LMM Output:
'The Prince' was written in 1513, when the Medici family had just regained power in Florence. In 1525, when Machiavelli dedicated the 'History of Florence' to the head of the family, its ruin was at hand.
------------------------------------------------------------------------------------------------------------------------
Question: What are the central themes and ideas presented in 'The Prince'?
LMM Output:
The central themes and ideas presented in 'The Prince' are the eternal problems between the ruled and their rulers, the ethics of Machiavelli's contemporaries, and the use of material rather than moral forces in government.
-------------------------------------------------------------------------------------------------------

In [20]:
print("------"*20)
question = "If I want to remain in power, what should I do?"
print("Question:", question)
docs = the_prince_vectorstore.similarity_search(question)
output = chain.run(input_documents = docs, question=question)
print("LMM Output:")
print(str.strip(output))
time.sleep(5)

------------------------------------------------------------------------------------------------------------------------
Question: If I want to remain in power, what should I do?
LMM Output:
You should win friends, overcome either by force or fraud, make yourself beloved and feared by the people, be followed and revered by the soldiers, exterminate those who have power or reason to hurt you, change the old order of things for new, be severe and gracious, magnanimous and liberal, destroy a disloyal soldiery and create new, and maintain friendship with kings and princes in such a way that they must help you with zeal.


### THE END